<h3><b>Train a Neural Machine Translation model using attention mechanism to enhance it's performance.</b></h3>

<h4><i>0. Setup </i></h4>

In [1]:
import os
import numpy as np
import tensorflow as tf

from model import Translator
from model import masked_loss, masked_acc 
from nmt_dataset import DatasetConfig
from nmt_dataset import load_dataset
from nmt_dataset import decode

<h4><i>1. Load dataset. </i></h4>

In [2]:
# define dataset loading config
ds_config = DatasetConfig(
    vocab_size=25_000,
    max_sequence_len=45,
    batch_size=32,
    inputs_preprocessor=None,
    targets_preprocessor=None,
)

In [3]:
# load dataset

print("loading dataset and tokenizers ...", end=" ")
dataset, inputs_vectorizer, targets_vectorizer = load_dataset(
    config=ds_config
)
print("done.")

loading dataset and tokenizers ... done.


In [4]:
# a quick glance at dataset's input and output structure.

for inputs, outputs in dataset.take(1):
    enc_input, dec_input = inputs["enc_inputs"], inputs["dec_inputs"]
    print(f"encoder input shape:{'':2} {enc_input.shape}")
    print(f"decoder input shape:{'':2} {dec_input.shape}")
    print(f"output target shape:{'':2} {outputs.shape}")

encoder input shape:   (32, 45)
decoder input shape:   (32, 44)
output target shape:   (32, 44)


<h4><i>2. Make a new model. </i></h4>

In [5]:
nmt_model = Translator(
    inputs_vectorizer,
    targets_vectorizer,
    units=512,
    attn_num_heads=6,
)

<h4><i>3. Compile and train the model. </i></h4>

In [7]:
# Compile the Model
nmt_model.compile(optimizer="adam", loss=masked_loss, metrics=[masked_loss, masked_acc])

In [8]:
# Fit the model.
history = nmt_model.fit(dataset, epochs=20)

Epoch 1/20
236/236 [==============================] - 514s 2s/step - loss: 6.1987 - masked_loss: 6.1977 - masked_acc: 0.1588
Epoch 2/20
236/236 [==============================] - 448s 2s/step - loss: 4.8464 - masked_loss: 4.8460 - masked_acc: 0.2754
Epoch 3/20
236/236 [==============================] - 448s 2s/step - loss: 3.6146 - masked_loss: 3.6151 - masked_acc: 0.3939
Epoch 4/20
236/236 [==============================] - 447s 2s/step - loss: 2.5346 - masked_loss: 2.5354 - masked_acc: 0.5028
Epoch 5/20
236/236 [==============================] - 460s 2s/step - loss: 1.5997 - masked_loss: 1.6002 - masked_acc: 0.6405
Epoch 6/20
236/236 [==============================] - 463s 2s/step - loss: 0.9485 - masked_loss: 0.9489 - masked_acc: 0.7691
Epoch 7/20
236/236 [==============================] - 458s 2s/step - loss: 0.5823 - masked_loss: 0.5821 - masked_acc: 0.8494
Epoch 8/20
236/236 [==============================] - 451s 2s/step - loss: 0.3750 - masked_loss: 0.3750 - masked_acc: 0.9020


<h4><i>4. Save the model. </i></h4>

In [ ]:
nmt_model.save("persian_to_english_nmt_model")

<h4><i> 5. Evaluate the model. </i></h4>

In [15]:
texts = ["بگو کجا ", "با تو موافقم", "من داشتم قدم می زدم", " من یک دکتر هستم"]

for t in texts:
    result = nmt_model.translate([t])
    translated = result[0].numpy().decode()
    print(f"{t:10}: {translated}")
    print()

بگو کجا   : say one tell 

با تو موافقم: you can be as you for a bit 

من داشتم قدم می زدم: i love and withered 

 من یک دکتر هستم: i enjoy a doctor 



It seems that the translation of quality is not satisfying. Some potential reason :
    1. Low quality dataset .
    2. Small number of samples.
    3. Training for just a few epochs.